In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

## Step 1: Setup & Data Load

In [2]:
# Load MiniLM model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Load datasets
pe_df = pd.read_excel("/Users/moaya/OneDrive/Desktop/mbd-corporateproject-EY-main/Data/Processed/Cleaned_PE.xlsx")
company_df = pd.read_excel("/Users/moaya/OneDrive/Desktop/mbd-corporateproject-EY-main/Data/Processed/outputcheckcombined2.xlsx")
transactions_df = pd.read_excel("/Users/moaya/OneDrive/Desktop/mbd-corporateproject-EY-main/Data/Processed/PE_Transactions.xlsx")

The sentence_transformers library is built on top of HuggingFace Transformers, optimized for creating dense semantic vector embeddings for sentences or phrases.

It’s a compact but powerful transformer model (MiniLM) that maps text (e.g. sectors, descriptions) into vector embeddings of fixed size.

We're using this model to:

* Convert sector names (e.g. "Pharmaceuticals", "Biotech") into numerical vectors.

* Then compute similarity between companies and PE firms based on how semantically close their sectors are.

## Step 2: Feature Preparation

In [3]:
# --- Normalize and Embed Sectors for Companies ---
def normalize_company_sectors(row):
    cols = ['1º_characterization', '2º_characterization', '3º_characterization',
            '4º_characterization', '5º_characterization', '6º_characterization']
    return [str(row[col]).strip().lower() for col in cols if pd.notna(row[col])]

company_df['normalized_sectors'] = company_df.apply(normalize_company_sectors, axis=1)
company_df['sector_embeddings'] = company_df['normalized_sectors'].apply(
    lambda sectors: sbert_model.encode(sectors, convert_to_tensor=True) if sectors else None
)

# --- Normalize and Embed Sectors for PE firms ---
pe_df['normalized_sectors'] = pe_df['sectors'].fillna('').apply(
    lambda x: [s.strip().lower() for s in x.split(',')]
)
pe_df['sector_embeddings'] = pe_df['normalized_sectors'].apply(
    lambda sectors: sbert_model.encode(sectors, convert_to_tensor=True) if sectors else None
)


# Merge again with normalized fields
pairs_df = company_df.merge(pe_df, how='cross')

# Fill missing Spanish company counts with 0
pairs_df['total_spanish_companies'] = pairs_df['total_spanish_companies'].fillna(0)


# Feature: Sector match
def embedding_sector_match_from_precomputed(row, threshold=0.6):
    cs_embeddings = row['sector_embeddings_x']
    ps_embeddings = row['sector_embeddings_y']
    if cs_embeddings is None or ps_embeddings is None:
        return 0
    cosine_scores = util.cos_sim(cs_embeddings, ps_embeddings)
    max_score = cosine_scores.max().item()
    return 1 if max_score >= threshold else 0

# Define function
def embedding_sector_similarity_score(row):
    cs_embeddings = row['sector_embeddings_x']
    ps_embeddings = row['sector_embeddings_y']
    if cs_embeddings is None or ps_embeddings is None:
        return 0
    cosine_scores = util.cos_sim(cs_embeddings, ps_embeddings)
    return cosine_scores.max().item()

# Compute similarity score
pairs_df['sector_similarity_score'] = pairs_df.apply(embedding_sector_similarity_score, axis=1)

# Use threshold to create binary match
pairs_df['sector_embedding_match'] = pairs_df['sector_similarity_score'] >= 0.6


# Feature: EBITDA in range
def ebitda_match(row):
    try:
        ebitda = float(row.get('ebitda_lya', None))
        min_val = row.get('ticket_ebitda_min', None)
        max_val = row.get('ticket_ebitda_max', None)

        if pd.isna(ebitda):
            return False

        # If both min and max are present
        if pd.notna(min_val) and pd.notna(max_val):
            return min_val <= ebitda <= max_val

        # If only min is present
        if pd.notna(min_val) and pd.isna(max_val):
            return ebitda >= min_val

        # If only max is present
        if pd.isna(min_val) and pd.notna(max_val):
            return ebitda <= max_val

        # If neither is present
        return False

    except:
        return False


pairs_df['ebitda_in_range'] = pairs_df.apply(ebitda_match, axis=1)

### Feature Engineering Explanation

This block performs key transformations and feature generation for matching companies with Private Equity (PE) firms:

---

#### 1. Normalize & Embed Company Sectors
- Collects up to 6 sector classifications (`1º_characterization` to `6º_characterization`) for each company.
- Cleans and lowers the text, storing it in `normalized_sectors`.
- Converts this list into vector embeddings using a pre-trained SentenceTransformer (`MiniLM`) for semantic comparison.

#### 2. Normalize & Embed PE Firm Sectors
- Extracts and cleans sector strings from the PE firms’ `sectors` column.
- Embeds them into semantic vectors using the same model.

#### 3. Create All Possible Company–PE Pairs
- A full Cartesian merge (`how='cross'`) is performed to create all combinations of companies and PE firms.
- This allows us to evaluate every possible match.

#### 4. Sector Similarity Score
- Computes the cosine similarity between company and PE firm sector embeddings.
- Stores the **maximum score** (most similar pair) in `sector_similarity_score`.

#### 5. EBITDA Compatibility
- `ebitda_in_range`: Checks whether the company's `ebitda_lya` is within the PE firm's `ticket_ebitda_min` to `ticket_ebitda_max`.
- This captures financial compatibility between the company and the PE firm's investment profile.
---

These engineered features will be used as model inputs to predict whether a PE firm is likely to be a good match for a given company.

## Step 3: Label: Known Match or Not

In [4]:
# Prepare a set of known (company, PE) pairs
known_matches = set(zip(transactions_df['Target'].str.lower().str.strip(),
                        transactions_df['PE'].str.lower().str.strip()))

# Assign target using fast set lookup
pairs_df['target'] = pairs_df.apply(
    lambda row: 1 if (row['company_name'], row['pe']) in known_matches else 0, axis=1
)

### Label Known Matches from Historical PE Transactions

This section assigns binary labels to each (company, PE firm) pair to indicate whether they are a known investment match based on past transaction data.

---

#### Step 1: Define Known Matches
We use the provided `transactions_df`, which includes actual historical deals between companies and PE firms. We:
- Normalize both company names (`Target`) and PE names by converting to lowercase and removing extra spaces.
- Combine them into a `set` of tuples for efficient lookup.

```python
known_matches = set(zip(transactions_df['Target'].str.lower().str.strip(),
                        transactions_df['PE'].str.lower().str.strip()))


In [5]:
# First, set all to 0
pairs_df['target'] = 0

# Simulate positive samples: randomly label N rows as 1
N_SIMULATED_MATCHES = 200  # You can adjust this

# Filter to rows that have sector match and EBITDA in range (more realistic)
candidate_matches = pairs_df[(pairs_df['sector_embedding_match'] == True) | 
                             (pairs_df['ebitda_in_range'] == True)]

# Randomly pick N rows to mark as matches
np.random.seed(42)
simulated_indices = candidate_matches.sample(n=min(N_SIMULATED_MATCHES, len(candidate_matches)), random_state=42).index
pairs_df.loc[simulated_indices, 'target'] = 1

# Check result
print("Simulated positive matches:", pairs_df['target'].sum())

Simulated positive matches: 200


### Simulating Positive Matches for Model Training

In this block, we artificially label a subset of (company, PE) pairs as **positive matches (`target = 1`)** to help train the machine learning model when there are too few known matches in the real data.

---

#### Why Simulate?

- The original dataset contains very few known company–PE matches (`target = 1`), making it **heavily imbalanced**.
- Most ML models perform poorly when trained on such skewed data.
- To address this, we simulate a **realistic set of matches** based on meaningful filters (sector or EBITDA compatibility).

In [6]:
# --- Create balanced training dataset ---
positives = pairs_df[pairs_df['target'] == 1]
negatives = pairs_df[pairs_df['target'] == 0]

# Keep all positives, sample equal size of negatives
neg_sampled = negatives.sample(n=len(positives), random_state=42)

# Final balanced dataframe
balanced_df = pd.concat([positives, neg_sampled]).sample(frac=1, random_state=42).reset_index(drop=True)

### Create a Balanced Training Dataset

Machine learning models often struggle with highly imbalanced data. Since the number of positive matches (`target = 1`) is much smaller than non-matches (`target = 0`), we create a **balanced dataset** for better training results.

What we did:

* Randomly selects an equal number of negative samples to match the count of positives.
* Ensures the model sees both classes equally during training.
* Combines positive and sampled negative samples.
* Shuffles the rows using sample(frac=1).
* Resets the index for a clean final DataFrame.

## Step 4: Train/Test Split and Model Training

In [7]:
features = ['ebitda_in_range', 'total_spanish_companies', 'sector_similarity_score']


X = balanced_df[features]
y = balanced_df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=42
)

### Train-Test Split for Model Training

After preparing a balanced dataset (`balanced_df`), we now split it into training and testing sets to evaluate the model's performance.

In [8]:
# Compute class imbalance ratio
imbalance_ratio = y_train.value_counts()[0] / y_train.value_counts()[1]

# Train XGBoost model
model = XGBClassifier(
    eval_metric='logloss',
    scale_pos_weight=imbalance_ratio,
    random_state=42
)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred_xgb = model.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.89      0.78      0.83        60
           1       0.81      0.90      0.85        60

    accuracy                           0.84       120
   macro avg       0.85      0.84      0.84       120
weighted avg       0.85      0.84      0.84       120



In [10]:
# === Predict Top PEs for a Specific Company ===

# Select company by name
selected_company = "Bioiberica"  # Change this to test other companies

# Get the company row
selected_company_row = company_df[company_df['company_name'].str.lower() == selected_company.lower()]
selected_company_row['key'] = 1
pe_df['key'] = 1

# Cross join with all PEs
recommend_df = selected_company_row.merge(pe_df, on='key').drop('key', axis=1)

recommend_df['total_spanish_companies'] = recommend_df['total_spanish_companies'].fillna(0)

recommend_df['sector_embeddings_x'] = recommend_df['normalized_sectors_x'].apply(
    lambda sectors: sbert_model.encode(sectors, convert_to_tensor=True) if sectors else None
)
recommend_df['sector_embeddings_y'] = recommend_df['normalized_sectors_y'].apply(
    lambda sectors: sbert_model.encode(sectors, convert_to_tensor=True) if sectors else None
)

# Recompute normalized sectors
recommend_df['normalized_sectors_x'] = recommend_df.apply(normalize_company_sectors, axis=1)
recommend_df['normalized_sectors_y'] = recommend_df['sectors'].fillna('').apply(lambda x: [s.strip().lower() for s in x.split(',')])

# Features
recommend_df['sector_similarity_score'] = recommend_df.apply(embedding_sector_similarity_score, axis=1)
recommend_df['sector_embedding_match'] = recommend_df['sector_similarity_score'].apply(lambda x: 1 if x >= 0.5 else 0)

# ✅ Updated EBITDA match using ebitda_lya
def ebitda_match(row):
    try:
        ebitda = float(row.get('ebitda_lya', None))
        min_val = row.get('ticket_ebitda_min', None)
        max_val = row.get('ticket_ebitda_max', None)

        if pd.isna(ebitda):
            return False

        # If both min and max are present
        if pd.notna(min_val) and pd.notna(max_val):
            return min_val <= ebitda <= max_val

        # If only min is present
        if pd.notna(min_val) and pd.isna(max_val):
            return ebitda >= min_val

        # If only max is present
        if pd.isna(min_val) and pd.notna(max_val):
            return ebitda <= max_val

        # If neither is present
        return False

    except:
        return False


recommend_df['ebitda_in_range'] = recommend_df.apply(ebitda_match, axis=1)


# Predict
X_recommend = recommend_df[[
    'ebitda_in_range',
    'total_spanish_companies',
    'sector_similarity_score'
]]

recommend_df['match_probability'] = model.predict_proba(X_recommend)[:, 1]


# Top output
top_matches = recommend_df[[
    'pe', 'match_probability', 'ebitda_in_range',
    'total_spanish_companies', 'sector_similarity_score'
]].sort_values(by='match_probability', ascending=False)

top_matches = top_matches.drop_duplicates(subset='pe')

top_matches.head(10)

C:\Users\moaya\AppData\Local\Temp\ipykernel_17040\511142322.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_company_row['key'] = 1


pe  match_probability  \
67                Bridgepoint Advisers Ltd           0.973677   
72                        CVC Advisers Ltd           0.961250   
82                             Permira Ltd           0.944231   
77                     InvestIndustrial LP           0.937477   
79                      Oakley Capital Ltd           0.937195   
118  Dea Capital Alternative Funds SGR SpA           0.924502   
110                       PAI Partners SAS           0.917455   
86                         TDR Capital LLP           0.902210   
62                       Apax Partners LLP           0.897647   
73                           HgCapital LLP           0.897647   

     ebitda_in_range  total_spanish_companies  sector_similarity_score  
67              True                        3                 0.245460  
72              True                       11                 0.308119  
82              True                        6                 0.363320  
77              True                        6                 0.240214  
79              True                        7                 0.218753  
118             True                        7                 0.444154  
110             True                        6                 0.444154  
86              True                        1                 0.255738  
62              True                        1                 0.392613  
73              True                        1                 0.392613